In [275]:
import re
from collections import Counter, defaultdict
from itertools import permutations
from math import prod
from typing import List
from functools import cache

import black
import jupyter_black
from parse import parse

jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)


def ints(text: str) -> List[int]:
    return [int(x) for x in re.findall("-?\d+", text)]


def first(iterable):
    return next(iter(iterable))

In [6]:
# Day 1: Report Repair
numbers = [int(x) for x in open("2020/1.txt").read().splitlines()]
for a, b in permutations(numbers, 2):
    if a + b == 2020:
        break
print(f"Part 1: {a*b}")  # 471019

for a, b, c in permutations(numbers, 3):
    if a + b + c == 2020:
        break
print(f"Part 2: {a*b*c}")  # 103927824

Part 1: 471019
Part 2: 103927824


In [24]:
# Day 2: Password Philosophy
def valid(line):
    lower, upper, char, password = parse("{:d}-{:d} {}: {}", line)
    return lower <= password.count(char) <= upper


def valid2(line):
    first, second, char, password = parse("{:d}-{:d} {}: {}", line)
    return ((password[first - 1] == char) + (password[second - 1] == char)) == 1


lines = open("2020/2.txt").read().splitlines()
print(f"Part 1: {sum(valid(line) for line in lines)}") # 398
print(f"Part 2: {sum(valid2(line) for line in lines)}") # 562

Part 1: 398
Part 2: 562


In [51]:
# Day 3: Toboggan Trajectory
def downhill(lines, right=3, down=1, tree="#"):
    return sum(
        1
        for row, line in enumerate(lines[::down])
        if line[(right * row) % len(line)] == tree
    )


def run(lines, slopes):
    return prod(downhill(lines, *slope) for slope in slopes)


lines = open("2020/3.txt").read().splitlines()
slopes = ((1, 1), (3, 1), (5, 1), (7, 1), (1, 2))
print(f"Part 1: {downhill(lines)}")  # 189
print(f"Part 2: {run(lines, slopes)}")  # 1718180100

Part 1: 189
Part 2: 1718180100


In [172]:
# Day 4: Passport Processing
def all_fields_present(passport):
    FIELDS_NEEDED = {"byr", "iyr", "eyr", "hgt", "hcl", "ecl", "pid"}
    fields = passport.split(" ")
    fields = {field.split(":")[0]: field.split(":")[1] for field in fields}
    return all(field in fields.keys() for field in FIELDS_NEEDED)


def valid(passport):
    def check_height(height):
        unit = height[-2:]
        value = first(ints(height))
        if unit == "cm":
            return 150 <= value <= 193
        elif unit == "in":
            return 59 <= value <= 76
        else:
            return False

    EYE_COLORS = {"amb", "blu", "brn", "grn", "gry", "hzl", "oth"}
    rules = {
        "byr": lambda year: 1920 <= int(year) <= 2002,
        "iyr": lambda year: 2010 <= int(year) <= 2020,
        "eyr": lambda year: 2020 <= int(year) <= 2030,
        "hgt": check_height,
        "hcl": lambda color: re.search("#[0-9a-f]{6}", color) is not None,
        "ecl": lambda color: color in EYE_COLORS,
        "pid": lambda number: re.search("^[0-9]{9}$", number) is not None,
        "cid": lambda x: True,
    }
    fields = passport.split(" ")
    fields = {field.split(":")[0]: field.split(":")[1] for field in fields}
    if all(rules[key](value) for key, value in fields.items()):
        return True


passports = open("2020/4.txt").read().strip().split("\n\n")
passports = [p.replace("\n", " ") for p in passports]
passports = [p for p in passports if all_fields_present(p)]
print(f"Part 1: {len(passports)}")
passports = [p for p in passports if valid(p)]
print(f"Part 2: {len(passports)}")

Part 1: 170
Part 2: 103


In [199]:
# Day 5: Binary Boarding
def seat_id(boarding_pass):
    row = boarding_pass[:7].replace("F", "0").replace("B", "1")
    column = boarding_pass[7:].replace("L", "0").replace("R", "1")
    return int(row, 2) * 8 + int(column, 2)


boarding_passes = open("2020/5.txt").read().splitlines()
seat_ids = sorted(seat_id(boarding_pass) for boarding_pass in boarding_passes)
print(f"Part 1: {max(seat_ids)}")
my_seat_id = first(
    (higher - 1) for lower, higher in zip(seat_ids, seat_ids[1:]) if higher - lower == 2
)
print(f"Part 2: {my_seat_id}")

Part 1: 991
Part 2: 534


In [41]:
# Day 6: Custom Customs
def unique_answers(groups):
    total = 0
    for group in groups:
        answers = set()
        for person in group:
            for question in person:
                answers.add(question)
        total += len(answers)
    return total


def all_yes_answers(groups):
    total = 0
    for group in groups:
        group_answers = Counter()
        for person in group:
            for question in person:
                group_answers += Counter(question)
        for answer, count in group_answers.items():
            if count == len(group):
                total += 1
    return total


groups = open("2020/6.txt").read().strip().split("\n\n")
groups = [group.splitlines() for group in groups]

print(f"Part 1: {unique_answers(groups)}")
print(f"Part 2: {all_yes_answers(groups)}")

Part 1: 6683
Part 2: 3122


In [194]:
# Day 7: Handy Haversacks
def parse_lines(lines):
    bags = defaultdict(lambda: list())
    parents = defaultdict(lambda: set())
    for line in lines:
        source, destinations = line.split(" bags contain ")
        for destination in destinations.split(", "):
            if p := parse("{:d} {} ba{}", destination):
                amount, bag, _ = p
                bags[source] += [(amount, bag)]
                parents[bag] |= {source}
            else:
                bags[source] = []

    return bags, parents


def can_contain(bag_type="shiny gold"):
    result = parents[bag_type].copy()
    for parent in parents[bag_type]:
        result |= can_contain(parent)
    return result


def number_of_bags(bag_type="shiny gold"):
    "Return number of bags contained in this bag"
    return sum(
        num_child + num_child * number_of_bags(child)
        for num_child, child in bags[bag_type]
    )


lines = open("2020/7.txt").read().splitlines()
bags, parents = parse_lines(lines)
print(f"Part 1: {len(can_contain())}")  # 197
print(f"Part 2: {number_of_bags()}")  # 85324

Part 1: 197
Part 2: 85324


In [238]:
# Day 8: Handheld Halting
def run_program(lines):
    "Returns (True, result) if program runs to completion, (False, result) if it ends up in an infinite loop"
    pc = acc = 0
    pcs = set()
    for _ in range(1000):
        instruction, value = parse("{} {:d}", lines[pc])
        match instruction:
            case "acc":
                acc += value
            case "jmp":
                pc += value - 1
        pc += 1
        if pc in pcs or pc >= len(lines):
            return (pc >= len(lines), acc)
        pcs.add(pc)


def modified_programs(lines):
    modified_lines = lines.copy()
    for pc, line in enumerate(lines):
        instruction, value = parse("{} {}", lines[pc])
        if instruction == "jmp":
            modified_lines[pc] = "nop " + value
        elif instruction == "nop":
            modified_lines[pc] = "jmp " + value
        else:
            continue
        yield modified_lines.copy()
        modified_lines[pc] = lines[pc]


def part2(lines):
    for program in modified_programs(lines):
        (part2, result) = run_program(program)
        if part2:
            return result


lines = open("2020/8.txt").read().splitlines()
print(f"Part 1: {run_program(lines)[1]}")  # 1675
print(f"Part 2: {part2(lines)}")  # 1532

Part 1: 1675
Part 2: 1532


In [282]:
# Day 9: Encoding Error
def valid(position, numbers, length):
    for a, b in permutations(numbers[position - length : position], 2):
        if numbers[position] == a + b:
            return True
    return False


def first_invalid(numbers, length=25):
    return first(
        numbers[pos]
        for pos in range(length, len(numbers))
        if not valid(pos, numbers, length)
    )


def target_sequence(target, numbers):
    for start in range(len(numbers)):
        for end in range(start + 1, len(numbers)):
            if sum(numbers[start:end]) == target:
                return numbers[start:end]


lines = open("2020/9.txt").read().splitlines()
numbers = [int(x) for x in lines]
invalid = first_invalid(numbers)
sequence = target_sequence(invalid, numbers)
print(f"Part 1: {invalid}")
print(f"Part 2: {min(sequence) + max(sequence)}")

Part 1: 57195069
Part 2: 7409241
